In [1]:
import pandas as pd 
sales = pd.read_csv('data/Transaction_data.csv', low_memory=False)
store = pd.read_csv('data/Store_Lookup.csv')
product = pd.read_csv('data/Product_Lookup.csv')

In [2]:
print(sales.head())  # View first 5 rows
print(sales.info()) 
print(sales.isnull().sum())

  WEEK_END_DATE  STORE_NUM         UPC  UNITS  VISITS  HHS  SPEND  PRICE  \
0     14-Jan-09        367  1111009477     13      13   13  18.07   1.39   
1     14-Jan-09        367  1111009497     20      18   18  27.80   1.39   
2     14-Jan-09        367  1111009507     14      14   14  19.32   1.38   
3     14-Jan-09        367  1111035398      4       3    3  14.00   3.50   
4     14-Jan-09        367  1111038078      3       3    3   7.50   2.50   

   BASE_PRICE  FEATURE  DISPLAY  TPR_ONLY  
0        1.57        0        0         1  
1        1.39        0        0         0  
2        1.38        0        0         0  
3        4.49        0        0         1  
4        2.50        0        0         0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 524950 entries, 0 to 524949
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   WEEK_END_DATE  524950 non-null  object 
 1   STORE_NUM      524950 non-nu

In [3]:
sales.columns = sales.columns.str.lower()

In [4]:
sales['week_end_date'] = pd.to_datetime(sales['week_end_date'])

C:\Users\JOSHUA\AppData\Local\Temp\ipykernel_5808\974956267.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sales['week_end_date'] = pd.to_datetime(sales['week_end_date'])


In [5]:
sales['week_end_date'] = pd.to_datetime(sales['week_end_date'], format='%d-%b-%y')

## CLEANING

In [6]:
coloums_to_check = ['store_num','upc','units','visits','hhs','spend','price','base_price']
print("Count of zeros in specified columns:")
for col in coloums_to_check:
    zero_count = (sales[col] == 0).sum()
    print(f" Zero's in {col}: {zero_count}")


Count of zeros in specified columns:
 Zero's in store_num: 0
 Zero's in upc: 0
 Zero's in units: 5
 Zero's in visits: 0
 Zero's in hhs: 0
 Zero's in spend: 24
 Zero's in price: 1
 Zero's in base_price: 0


==================================

count of 0's and null values: 

Zero's in units: 5

Zero's in spend: 24

Zero's in price: 1

null values in price: 23

null values in base_price: 185

============================================

Data Cleaning Plan for Units = 0 

if units = 0 & hhs = visits , then units = visits

if units = 0 & hhs < visits, then units = visits

if units = 0 & hhs > visit, then units = hhs

In [7]:
sales.loc[(sales['units'] == 0) & (sales['hhs'] == sales['visits']), 'units'] = sales['visits']
sales.loc[(sales['units'] == 0) & (sales['hhs'] < sales['visits']), 'units'] = sales['visits']
sales.loc[(sales['units'] == 0) & (sales['hhs'] > sales['visits']), 'units'] = sales['hhs']

Filling missing/ 0 base_price:

if tpr = 1, check the upc and median the baseprice

if tpr = 0 , then base price = spend/units

In [10]:
# CASE 1: If base_price is NaN or 0, and TPR = 0 ; base price = spend / units
mask_base_invalid = (sales['base_price'].isnull()) | (sales['base_price'] == 0)
mask_tpr_0 = sales['tpr_only'] == 0
sales.loc[mask_base_invalid & mask_tpr_0, 'base_price'] = sales['spend'] / sales['units']

# CASE 2: If base_price is still NaN or 0 ; base_price = median base price for that UPC
mask_still_invalid = (sales['base_price'].isnull()) | (sales['base_price'] == 0)
median_base = sales.groupby('upc')['base_price'].median()
sales.loc[mask_still_invalid, 'base_price'] = sales.loc[mask_still_invalid, 'upc'].map(median_base)

Filling missing/0 price:

if tpr = 1, check the upc and median the baseprice

if tpr = 0 , then price = base price

In [13]:
# CASE 1: If TPR = 0 → price = base_price
mask_base_invalid = (sales['price'].isnull()) | (sales['price'] == 0)
mask_tpr_0 = sales['tpr_only'] == 0
sales.loc[mask_base_invalid & mask_tpr_0, 'price'] = sales['spend'] / sales['units']

# CASE 2: If TPR = 1, fill from average of price for the same UPC
mask_base_invalid = (sales['price'].isnull()) | (sales['price'] == 0)
mean_price = sales.groupby('upc')['price'].mean()
sales.loc[mask_still_invalid, 'price'] = sales.loc[mask_still_invalid, 'upc'].map(median_base)

## FLAGGING OUTLIERS
1. Are customers buying too many units per trip?
    -- Flagging bulk-buying or data errors
2. Are there too many visits per household in a week?
    -- Anomolies /over purchasing
 

In [14]:
sales['units_per_visit'] = sales['units'] / sales['visits']
sales['visits_per_hhs'] = sales['visits'] / sales['hhs']

In [15]:
# 95th percentile for outliers
units_per_visit_threshold = sales['units_per_visit'].quantile(0.95)
visits_per_hhs_threshold = sales['visits_per_hhs'].quantile(0.95)

print(f"95th percentile units/visit: {units_per_visit_threshold:.2f}")
print(f"95th percentile visits/hhs: {visits_per_hhs_threshold:.2f}")

95th percentile units/visit: 1.50
95th percentile visits/hhs: 1.10


In [16]:
sales['outlier_flag'] = (
    (sales['units_per_visit'] > units_per_visit_threshold) |
    (sales['visits_per_hhs'] > visits_per_hhs_threshold)
)

In [17]:
# Count flagged outliers
print(f"Outliers flagged: {sales['outlier_flag'].sum()} / {len(sales)} rows")

sales[sales['outlier_flag']].head()

Outliers flagged: 44502 / 524950 rows


,week_end_date,store_num,upc,units,visits,hhs,spend,price,base_price,feature,display,tpr_only,units_per_visit,visits_per_hhs,outlier_flag
13,2009-01-14,367,1600027528,31,26,19,142.29,4.59,4.59,0,0,0,1.192308,1.368421,True
14,2009-01-14,367,1600027564,56,48,42,152.32,2.72,3.07,1,0,0,1.166667,1.142857,True
21,2009-01-14,367,3800031829,14,11,3,43.96,3.14,3.14,0,0,0,1.272727,3.666667,True
25,2009-01-14,367,7027316204,13,8,8,26.26,2.02,3.29,0,0,1,1.625000,1.000000,True
52,2009-01-14,387,2840004768,21,19,17,60.69,2.89,2.89,0,0,0,1.105263,1.117647,True


In [18]:
outliers_df = sales[sales['outlier_flag']]
print(outliers_df.sort_values(by='visits_per_hhs', ascending=False).head())

       week_end_date  store_num          upc  units  visits  hhs  spend  \
479987    2011-10-05       8035  88491201426     24      20    1  83.52   
496432    2011-11-09       8035  88491201426     18      12    1  63.72   
390404    2011-04-06       8035  88491201426     26      21    2  81.12   
50053     2009-05-06       4259  88491201426     23      19    2  59.34   
224601    2010-05-05      23055  88491201426     25      18    2  86.25   

        price  base_price  feature  display  tpr_only  units_per_visit  \
479987   3.48        3.48        0        0         0         1.200000   
496432   3.54        3.54        0        0         0         1.500000   
390404   3.12        3.12        0        0         0         1.238095   
50053    2.58        2.58        0        0         0         1.210526   
224601   3.45        3.45        0        0         0         1.388889   

        visits_per_hhs  outlier_flag  
479987            20.0          True  
496432            12.0    

In [19]:
print(outliers_df.sort_values(by='units_per_visit', ascending=False).head())

       week_end_date  store_num         upc  units  visits  hhs   spend  \
323827    2010-11-24       6179  3700031613     51       2    2  152.49   
327963    2010-12-01      13827  1111009477    480      25   21  542.40   
324470    2010-11-24      13827  1111009477    670      39   31  750.40   
292912    2010-09-22      13827  1111009477    422      36   32  476.86   
331454    2010-12-08      13827  1111009477    519      45   40  581.28   

        price  base_price  feature  display  tpr_only  units_per_visit  \
323827   2.99        4.29        1        0         0        25.500000   
327963   1.13        1.13        0        0         0        19.200000   
324470   1.12        1.12        0        0         0        17.179487   
292912   1.13        1.14        0        1         0        11.722222   
331454   1.12        1.12        0        0         0        11.533333   

        visits_per_hhs  outlier_flag  
323827        1.000000          True  
327963        1.190476    

In [20]:
sales['discount'] = sales['base_price'] - sales['price']
sales['discount_pct'] = ((sales['discount']*100) / sales['base_price']).round(3)

In [21]:
sales.sort_values(by="discount_pct", ascending=False).head(5)

,week_end_date,store_num,upc,units,visits,hhs,spend,price,base_price,feature,display,tpr_only,units_per_visit,visits_per_hhs,outlier_flag,discount,discount_pct
304180,2010-10-13,19533,7218063983,1,1,1,0.0,0.0,7.29,0,0,0,1.0,1.0,False,7.29,100.0
325502,2010-11-24,21485,2840002333,1,1,1,0.0,0.0,2.89,0,0,0,1.0,1.0,False,2.89,100.0
150740,2009-12-02,28909,31254742735,2,2,2,0.0,0.0,4.99,0,0,0,1.0,1.0,False,4.99,100.0
481806,2011-10-05,25021,3000006610,1,1,1,0.0,0.0,3.09,0,0,0,1.0,1.0,False,3.09,100.0
246398,2010-06-23,367,7027316404,3,3,3,0.0,0.0,1.97,0,0,1,1.0,1.0,False,1.97,100.0


In [22]:
print(store.head())
print(store.info()) 
print(store.isnull().sum())

   STORE_ID          STORE_NAME ADDRESS_CITY_NAME ADDRESS_STATE_PROV_CODE  \
0       389          SILVERLAKE          ERLANGER                      KY   
1      2277  ANDERSON TOWNE CTR        CINCINNATI                      OH   
2      4259       WARSAW AVENUE        CINCINNATI                      OH   
3      6379            KINGWOOD          KINGWOOD                      TX   
4      6431        AT WARD ROAD           BAYTOWN                      TX   

   MSA_CODE SEG_VALUE_NAME  PARKING_SPACE_QTY  SALES_AREA_SIZE_NUM  \
0     17140     MAINSTREAM              408.0                46073   
1     17140        UPSCALE                NaN                81958   
2     17140          VALUE                NaN                48813   
3     26420     MAINSTREAM                NaN                50237   
4     26420          VALUE              350.0                43698   

   AVG_WEEKLY_BASKETS  
0               24767  
1               54053  
2               31177  
3               2062

In [23]:
store.columns = store.columns.str.lower()

In [24]:
print(product.head())
print(product.info()) 
print(product.isnull().sum())

          UPC                DESCRIPTION   MANUFACTURER  \
0  1111009477     PL MINI TWIST PRETZELS  PRIVATE LABEL   
1  1111009497          PL PRETZEL STICKS  PRIVATE LABEL   
2  1111009507          PL TWIST PRETZELS  PRIVATE LABEL   
3  3500068914        COLG SPEARMINT WISP        COLGATE   
4  3700019521  CREST PH WHTG CLN MINT TP          P & G   

                CATEGORY                 SUB_CATEGORY PRODUCT_SIZE  
0             BAG SNACKS                     PRETZELS        15 OZ  
1             BAG SNACKS                     PRETZELS        15 OZ  
2             BAG SNACKS                     PRETZELS        15 OZ  
3  ORAL HYGIENE PRODUCTS  MOUTHWASH/RINSES AND SPRAYS        16 CT  
4  ORAL HYGIENE PRODUCTS  MOUTHWASH/RINSES AND SPRAYS       4.2 OZ  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   UPC           58 non-null     int64 
 1 

In [25]:
product.columns = product.columns.str.lower()

In [26]:
sales['store_num'] = sales['store_num'].astype(str)
sales['upc'] = sales['upc'].astype(str)
store['store_id'] = store['store_id'].astype(str)
store['msa_code'] = store['msa_code'].astype(str)
product['upc'] = product['upc'].astype(str)

In [ ]:
outliers_df.to_csv("sales_outliers.csv", index=False)

In [ ]:
sales.to_csv("cleaned_sales.csv", index=False)

## EDA

Total Rows, Unique Weeks, Products, Stores

In [27]:
print("Total transactions:", len(sales))
print("Date range:", sales['week_end_date'].min(), "to", sales['week_end_date'].max())
print("Total weeks:", sales['week_end_date'].nunique())

print("Unique products (UPC):", sales['upc'].nunique())
print("Unique stores:", sales['store_num'].nunique())

Total transactions: 524950
Date range: 2009-01-14 00:00:00 to 2012-01-04 00:00:00
Total weeks: 156
Unique products (UPC): 55
Unique stores: 77


Category, Brand, and Manufacturer

In [30]:
sales_product = pd.merge(sales, product, on='upc', how='left')

In [32]:
print("Unique categories:", sales_product['category'].nunique())
print("Unique subcategories:", sales_product['sub_category'].nunique())
print("Unique manufacturers:", sales_product['manufacturer'].nunique())

# category count
print("Category Counts:\n",sales_product['category'].value_counts())

Unique categories: 4
Unique subcategories: 7
Unique manufacturers: 16
Category Counts:
 category
COLD CEREAL              169689
BAG SNACKS               127462
ORAL HYGIENE PRODUCTS    116529
FROZEN PIZZA             111270
Name: count, dtype: int64


In [33]:
full_data = pd.merge(sales_product, store, left_on='store_num', right_on='store_id', how='left')

print("Store segment distribution:\n", full_data['seg_value_name'].value_counts())


Store segment distribution:
 seg_value_name
MAINSTREAM    301807
UPSCALE       121829
VALUE         115007
Name: count, dtype: int64


In [34]:
product_coverage = sales.groupby('upc')['week_end_date'].nunique().reset_index()
product_coverage.columns = ['upc', 'weeks_available']
print(product_coverage.describe())

store_coverage = sales.groupby('store_num')['week_end_date'].nunique().reset_index()
store_coverage.columns = ['store_num', 'weeks_active']
print(store_coverage.describe())


       weeks_available
count        55.000000
mean        150.981818
std          14.921388
min          98.000000
25%         156.000000
50%         156.000000
75%         156.000000
max         156.000000
       weeks_active
count     77.000000
mean     155.675325
std        2.849014
min      131.000000
25%      156.000000
50%      156.000000
75%      156.000000
max      156.000000


### Sales Overview

In [37]:
print("Total units sold:", sales['units'].sum())
print("Total dollar spend:", sales['spend'].sum().round(2))
print("Total number of baskets (visits):", sales['visits'].sum())


Total units sold: 10293368
Total dollar spend: 27927722.58
Total number of baskets (visits): 9012000


In [38]:
print("Date range:", sales['week_end_date'].min(), "to", sales['week_end_date'].max())
print("Number of weeks:", sales['week_end_date'].nunique())


Date range: 2009-01-14 00:00:00 to 2012-01-04 00:00:00
Number of weeks: 156


In [39]:
print("Unique products (UPC):", sales['upc'].nunique())
print("Unique stores:", sales['store_num'].nunique())


Unique products (UPC): 55
Unique stores: 77


### Category Overview

In [44]:
merged = sales.merge(product, on='upc', how='left')

category_summary = merged.groupby('category').agg({
    'units': 'sum',
    'spend': 'sum',
    'upc': pd.Series.nunique
}).rename(columns={'upc': 'Unique_Products'}).sort_values(by='spend', ascending=False)

print(category_summary)

                         units        spend  Unique_Products
category                                                    
COLD CEREAL            5827388  15008351.27               15
FROZEN PIZZA           1343294   6459557.59               12
BAG SNACKS             2601459   4731982.53               15
ORAL HYGIENE PRODUCTS   521227   1727831.19               13


### Store Overview

In [45]:
merged_store = sales.merge(store, left_on='store_num', right_on='store_id', how='left')

tier_summary = merged_store.groupby('seg_value_name').agg({
    'spend': 'sum',
    'units': 'sum',
    'store_num': pd.Series.nunique
}).rename(columns={'store_num': 'unique_stores'})

print(tier_summary)


                      spend    units  unique_stores
seg_value_name                                     
MAINSTREAM      15814325.76  5921002             43
UPSCALE          7790323.60  2825748             17
VALUE            5090707.00  1823254             19


### Price Distribution by Product

In [46]:
# Average and max price per product
product_price_stats = sales.groupby('upc').agg(
    avg_price=('price', 'mean'),
    max_price=('price', 'max'),
    base_price=('base_price', 'mean'),
    avg_discount_pct=('discount_pct', 'mean'),
    count_obs=('price', 'count')
).reset_index()

# Join with product lookup
product_price_stats = product_price_stats.merge(product[['upc', 'description', 'manufacturer', 'category']], on='upc', how='left')

# View top 10 most expensive products
product_price_stats.sort_values(by='avg_price', ascending=False).head(10)


,upc,avg_price,max_price,base_price,avg_discount_pct,count_obs,description,manufacturer,category
27,3500068914,7.496004,7.99,7.871880,4.827443,1399,COLG SPEARMINT WISP,COLGATE,ORAL HYGIENE PRODUCTS
46,7218063052,6.314345,8.91,6.880379,8.094223,10404,FRSC BRCK OVN ITL PEP PZ,TONYS,FROZEN PIZZA
48,7218063983,6.298191,7.89,6.880730,8.315577,8711,FRSC 4 CHEESE PIZZA,TONYS,FROZEN PIZZA
47,7218063979,6.296792,7.89,6.878320,8.308034,9755,FRSC PEPPERONI PIZZA,TONYS,FROZEN PIZZA
17,2066200532,6.025118,6.99,6.209281,2.861437,2755,NWMN OWN SUPREME PIZZA,KING,FROZEN PIZZA
15,2066200530,6.015117,6.99,6.172510,2.464374,5107,NWMN OWN PEPPERONI PIZZA,KING,FROZEN PIZZA
16,2066200531,6.009884,6.99,6.164991,2.423673,3807,NWMN OWN 4 CHEESE PIZZA,KING,FROZEN PIZZA
45,7192100339,5.981547,7.49,6.554559,8.514268,11962,DIGRN PEPP PIZZA,TOMBSTONE,FROZEN PIZZA
44,7192100337,5.978633,7.49,6.555917,8.575327,11918,DIGRN SUPREME PIZZA,TOMBSTONE,FROZEN PIZZA
43,7192100336,5.970749,7.49,6.559114,8.732991,11796,DIGIORNO THREE MEAT,TOMBSTONE,FROZEN PIZZA


### Most Frequently Discounted Products

In [48]:
# How many times each product was on discount
discount_freq = sales[sales['discount_pct'] > 0].groupby('upc').size().reset_index(name='discounted_times')

# Join with total product appearances to get % of times discounted
product_total_counts = sales.groupby('upc').size().reset_index(name='total_times')
discount_stats = discount_freq.merge(product_total_counts, on='upc')
discount_stats['discount_freq_pct'] = round(discount_stats['discounted_times'] / discount_stats['total_times'] * 100, 2)

# Merge with product info
discount_stats = discount_stats.merge(product[['upc', 'description', 'manufacturer', 'category']], on='upc')

# Top 10 most frequently discounted
discount_stats.sort_values(by='discount_freq_pct', ascending=False).head(10)


,upc,discounted_times,total_times,discount_freq_pct,description,manufacturer,category
37,7027312504,2489,4546,54.75,SHURGD PRETZEL RODS,SHULTZ,BAG SNACKS
39,7027316404,2746,5051,54.37,SHURGD PRETZEL STICKS,SHULTZ,BAG SNACKS
38,7027316204,2691,5059,53.19,SHURGD MINI PRETZELS,SHULTZ,BAG SNACKS
28,3700019521,4233,8949,47.30,CREST PH WHTG CLN MINT TP,P & G,ORAL HYGIENE PRODUCTS
14,1600027564,5561,11980,46.42,GM CHEERIOS,GENERAL MI,COLD CEREAL
5,1111038080,4433,10673,41.53,PL ANTSPTC SPG MNT MTHWS,PRIVATE LABEL,ORAL HYGIENE PRODUCTS
4,1111038078,4660,11249,41.43,PL BL MINT ANTSPTC RINSE,PRIVATE LABEL,ORAL HYGIENE PRODUCTS
43,7192100336,4548,11796,38.56,DIGIORNO THREE MEAT,TOMBSTONE,FROZEN PIZZA
44,7192100337,4511,11918,37.85,DIGRN SUPREME PIZZA,TOMBSTONE,FROZEN PIZZA
45,7192100339,4507,11962,37.68,DIGRN PEPP PIZZA,TOMBSTONE,FROZEN PIZZA


### Category & Brand-Level Price Behavior

#### Price & Discount Stats by Category

In [49]:
category_price_summary = sales.merge(product[['upc', 'category']], on='upc') \
    .groupby('category').agg(
        avg_price=('price', 'mean'),
        avg_base_price=('base_price', 'mean'),
        avg_discount_pct=('discount_pct', 'mean'),
        discount_freq=('discount_pct', lambda x: (x > 0).sum() / len(x))
    ).reset_index()

print(category_price_summary.sort_values(by='avg_discount_pct', ascending=False))


                category  avg_price  avg_base_price  avg_discount_pct  \
3  ORAL HYGIENE PRODUCTS   3.765057        4.026778          7.451257   
2           FROZEN PIZZA   5.313967        5.727637          6.501931   
1            COLD CEREAL   2.739635        2.889042          4.965756   
0             BAG SNACKS   2.200540        2.310110          4.466812   

   discount_freq  
3       0.287293  
2       0.283661  
1       0.227369  
0       0.244912  


#### Brand Discount Rankings (within category)

In [50]:
brand_discount = sales.merge(product[['upc', 'category', 'manufacturer']], on='upc') \
    .groupby(['category', 'manufacturer']).agg(
        avg_discount_pct=('discount_pct', 'mean'),
        times_discounted=('discount_pct', lambda x: (x > 0).sum()),
        total_observations=('discount_pct', 'count')
    ).reset_index()

brand_discount['discount_frequency'] = round(brand_discount['times_discounted'] / brand_discount['total_observations'] * 100, 2)

# Top discounted brands per category
brand_discount.sort_values(['category', 'discount_frequency'], ascending=[True, False]).head(15)


,category,manufacturer,avg_discount_pct,times_discounted,total_observations,discount_frequency
3,BAG SNACKS,SHULTZ,13.792032,7926,14656,54.08
4,BAG SNACKS,SNYDER S,5.456464,10344,31754,32.58
2,BAG SNACKS,PRIVATE LABEL,2.490033,7396,35188,21.02
1,BAG SNACKS,MKSL,3.152682,2682,13736,19.53
0,BAG SNACKS,FRITO LAY,1.961644,2869,32128,8.93
9,COLD CEREAL,QUAKER,6.870515,8373,30900,27.10
5,COLD CEREAL,GENERAL MI,6.602340,9522,35948,26.49
8,COLD CEREAL,PRIVATE LABEL,3.185598,8410,35902,23.42
6,COLD CEREAL,KELLOGG,5.094588,7959,35832,22.21
7,COLD CEREAL,POST FOODS,3.088553,4318,31107,13.88


#### Category-Level Price & Discount Behavior

In [53]:
category_price_summary = sales.merge(product[['upc', 'category']], on='upc') \
    .groupby('category').agg(
        avg_price=('price', 'mean'),
        avg_base_price=('base_price', 'mean'),
        avg_discount_pct=('discount_pct', 'mean'),
        discount_freq=('discount_pct', lambda x: (x > 0).sum() / len(x))
    ).reset_index()

category_price_summary = category_price_summary.sort_values(by='avg_discount_pct', ascending=False)
print(category_price_summary)

                category  avg_price  avg_base_price  avg_discount_pct  \
3  ORAL HYGIENE PRODUCTS   3.765057        4.026778          7.451257   
2           FROZEN PIZZA   5.313967        5.727637          6.501931   
1            COLD CEREAL   2.739635        2.889042          4.965756   
0             BAG SNACKS   2.200540        2.310110          4.466812   

   discount_freq  
3       0.287293  
2       0.283661  
1       0.227369  
0       0.244912  


In [ ]:
category_price_summary.to_csv("category_summary.csv", index=False)

 #### Brand-Level Discount Behavior (within each category)

In [54]:
brand_discount = sales.merge(product[['upc', 'category', 'manufacturer']], on='upc') \
    .groupby(['category', 'manufacturer']).agg(
        avg_discount_pct=('discount_pct', 'mean'),
        times_discounted=('discount_pct', lambda x: (x > 0).sum()),
        total_observations=('discount_pct', 'count')
    ).reset_index()

brand_discount['discount_frequency'] = round(
    brand_discount['times_discounted'] / brand_discount['total_observations'] * 100, 2
)

brand_discount_sorted = brand_discount.sort_values(
    ['category', 'discount_frequency'], ascending=[True, False]
)

print(brand_discount_sorted.head(20))


                 category   manufacturer  avg_discount_pct  times_discounted  \
3              BAG SNACKS         SHULTZ         13.792032              7926   
4              BAG SNACKS       SNYDER S          5.456464             10344   
2              BAG SNACKS  PRIVATE LABEL          2.490033              7396   
1              BAG SNACKS           MKSL          3.152682              2682   
0              BAG SNACKS      FRITO LAY          1.961644              2869   
9             COLD CEREAL         QUAKER          6.870515              8373   
5             COLD CEREAL     GENERAL MI          6.602340              9522   
8             COLD CEREAL  PRIVATE LABEL          3.185598              8410   
6             COLD CEREAL        KELLOGG          5.094588              7959   
7             COLD CEREAL     POST FOODS          3.088553              4318   
12           FROZEN PIZZA      TOMBSTONE          8.606985             13566   
13           FROZEN PIZZA          TONYS

In [55]:
brand_discount_sorted.to_csv("brand_discount_summary.csv", index=False)

In [57]:
weekly_sales = sales.groupby("week_end_date")[["units", "spend"]].sum().reset_index()
weekly_sales.to_csv("weekly_sales.csv", index=False)

In [58]:
sales_cat = (
    sales.merge(product, on="upc")
    .groupby("category")[["units", "spend"]]
    .sum()
    .reset_index()
)
sales_cat.to_csv("category_sales.csv", index=False)

In [59]:
sales_brand = (
    sales.merge(product, on="upc")
    .groupby("manufacturer")[["units", "spend"]]
    .sum()
    .reset_index()
    .sort_values(by="spend", ascending=False)
    .head(20)
)
sales_brand.to_csv("top_brands_sales.csv", index=False)

In [60]:
category_discount = (
    sales.merge(product, on="upc")
    .groupby("category")["discount_pct"]
    .mean()
    .reset_index()
    .sort_values(by="discount_pct", ascending=False)
)
category_discount.to_csv("category_discounts.csv", index=False)

In [61]:
brand_discount = (
    sales.merge(product, on="upc")
    .groupby("manufacturer")["discount_pct"]
    .mean()
    .reset_index()
    .sort_values(by="discount_pct", ascending=False)
    .head(20)
)
brand_discount.to_csv("brand_discounts.csv", index=False)

In [62]:
sales_store = (
    sales.merge(store, left_on="store_num", right_on="store_id")
    .groupby("seg_value_name")[["units", "spend"]]
    .sum()
    .reset_index()
)
sales_store.to_csv("store_segment_sales.csv", index=False)

In [63]:
sales_cat_tier = (
    sales.merge(product, on="upc")
    .merge(store, left_on="store_num", right_on="store_id")
    .groupby(["category", "seg_value_name"])[["units", "spend"]]
    .sum()
    .reset_index()
)
sales_cat_tier.to_csv("category_store_tier_matrix.csv", index=False)